# Advent of Code: Day 8

In [188]:
using Base.Iterators
using DataStructures
using JuMP
using GLPK
using Match
using Underscores

#fn = "ExampleInput.txt"
fn = "SolutionInput.txt"

p(ln) = begin
  i,o = split(ln, " | ")
  (i=split(i, " "),o=split(o, " "))
end

D = @_ readlines(fn) |>
       map(p, __)    |>
       collect

200-element Vector{NamedTuple{(:i, :o), Tuple{Vector{SubString{String}}, Vector{SubString{String}}}}}:
 (i = ["dfgabce", "cadfgb", "cefa", "ca", "aecbg", "dfcegb", "geabd", "ecbfg", "cab", "agcfbe"], o = ["egbfadc", "dbgae", "gcfeb", "abgdfc"])
 (i = ["fbcag", "eg", "dbge", "gcbfe", "acdgfe", "gec", "fdcegba", "efbdc", "fbedgc", "efabdc"], o = ["ge", "fagbc", "dfebc", "eg"])
 (i = ["da", "gdbcea", "gdcfbe", "acgbd", "dbfcgea", "ebad", "aecdfg", "afcbg", "adc", "ecdbg"], o = ["afcegd", "dacebg", "ad", "cdbafeg"])
 (i = ["gafeb", "ag", "acfbed", "ega", "ebadfg", "bfcge", "dgfa", "begcda", "edfba", "cfgedba"], o = ["ag", "dafg", "dfcbgea", "feacdb"])
 (i = ["cefdag", "gcea", "fag", "defca", "gbdaef", "afcdg", "dcbgf", "ga", "adfecbg", "fcbdea"], o = ["geac", "cfedba", "cgea", "fadecg"])
 (i = ["fcgaed", "abdecgf", "cfaed", "ceb", "febacd", "dbecf", "bc", "cbfa", "gecbad", "fbdeg"], o = ["facde", "cfgade", "acdefb", "cbe"])
 (i = ["bcgae", "cfedba", "efcbadg", "dafbc", "dfbe", "fcgbad", "e

## Part 1

In [189]:
r0 = @_ D                    |>
     map(_.o, __)            |> 
     map(length.(_), __)     |>
     filter.(_∈[2,4,3,7],__) |>
     map(length(_),__)       |>
     sum

440

## Part 2

In [190]:
K = [[1,1,1,1,1,1,0],#0
     [1,1,0,0,0,0,0],#1
     [1,0,1,1,0,1,1],#2
     [1,1,1,0,0,1,1],#3
     [1,1,0,0,1,0,1],#4
     [0,1,1,0,1,1,1],#5
     [0,1,1,1,1,1,1],#6
     [1,1,0,0,0,1,0],#7
     [1,1,1,1,1,1,1],#8
     [1,1,1,0,1,1,1]]#9

translate(I,O) = begin
  M = Model(GLPK.Optimizer)

  @variable(M,x[n=1:7,p=1:7],Bin)
  @variable(M,y[n=1:7,v=0:9],Bin)

  # Each variable cannot have the same position as another variable
  for p ∈ 1:7
    @constraint(M,sum(x[n,p] for n ∈ 1:7)==1)
  end

  @constraint(M,sum(y[n,0] for n ∈ 1:7)==6)
  @constraint(M,sum(y[n,1] for n ∈ 1:7)==2)
  @constraint(M,sum(y[n,2] for n ∈ 1:7)==5)
  @constraint(M,sum(y[n,3] for n ∈ 1:7)==5)
  @constraint(M,sum(y[n,4] for n ∈ 1:7)==4)
  @constraint(M,sum(y[n,5] for n ∈ 1:7)==5)
  @constraint(M,sum(y[n,6] for n ∈ 1:7)==6)
  @constraint(M,sum(y[n,7] for n ∈ 1:7)==3)
  @constraint(M,sum(y[n,8] for n ∈ 1:7)==7)
  @constraint(M,sum(y[n,9] for n ∈ 1:7)==6)

  for n ∈ 1:7
    # Each variable can only have 1 position
    @constraint(M,sum(x[n,p] for p ∈ 1:7)==1)

    # If a variable has value 0 then it cannot have position 7
    @constraint(M, y[n,0] <= 1 - x[n,7])

    # If a variable has value 1 then it cannot have positions 3-7
    for p ∈ 3:7
      @constraint(M, y[n,1] <= 1 - x[n,p])
    end

    # If a variable has value 2 then it cannot have positions 2 or 5
    for p ∈ [2,5]
      @constraint(M, y[n,2] <= 1 - x[n,p])
    end

    # If a variable has value 3 then it cannot have positions 4 or 5
    for p ∈ [4,5]
      @constraint(M, y[n,3] <= 1 - x[n,p])
    end

    # If a variable has value 4 then it cannot have positions 3, 4, or 6
    for p ∈ [3,4,6]
      @constraint(M, y[n,4] <= 1 - x[n,p])
    end

    # If a variable has value 5 then it cannot have positions 1 or 4
    for p ∈ [1,4]
      @constraint(M, y[n,5] <= 1 - x[n,p])
    end

    # If a variable has value 6 then it cannot have positions 1
    @constraint(M, y[n,6] <= 1 - x[n,1])

    # If a variable has value 7 then it cannot have positions 3, 4, 5, or 7
    for p ∈ [3,4,5,7]
      @constraint(M, y[n,7] <= 1 - x[n,p])
    end

    # If a variable has value 9 then it cannot have position 4
    @constraint(M, y[n,9] <= 1 - x[n,4])
  end

  encode_var(c) = @match c begin
    'a' => 1
    'b' => 2
    'c' => 3
    'd' => 4
    'e' => 5
    'f' => 6
    'g' => 7
  end

  encode_val(r) = @match length(r) begin
    2 => 1
    3 => 7
    4 => 4
    7 => 8
  end

  # Constaints based on input signals
  for i ∈ I
    V = map(encode_var, collect(i))
    for n ∈ V
      @match length(i) begin
        2 => @constraint(M, y[n,1]==1);
        3 => @constraint(M, y[n,7]==1);
        4 => @constraint(M, y[n,4]==1);
        7 => @constraint(M, y[n,8]==1);
      end
    end
  end

  I5 = filter(i->length(i)==5,I)
  for v ∈ ['a','b','c','d','e','f','g']
    t = count(i->occursin(v,i),I5)
    if t > 0
      n = encode_var(v)
      @constraint(M, y[n,2]+y[n,3]+y[n,5]==t)
    end
  end
  I6 = filter(i->length(i)==6,I)
  for v ∈ ['a','b','c','d','e','f','g']
    t = count(i->occursin(v,i),I6)
    if t > 0
      n = encode_var(v)
      @constraint(M, y[n,0]+y[n,6]+y[n,9]==t)
    end
  end

  optimize!(M)

  S = @_ value.(x) |> findall(_==1,__)
  Di = @_ S |> map(Tuple(_), __) |> Dict
  D = Dict()

  for (i, c) ∈ enumerate(['a','b','c','d','e','f','g'])
    D[Di[i]] = c
  end

  Ko = Dict()
  for (v,k) ∈ enumerate(K)
    k_t = []
    for k_i ∈ 1:length(k)
      if k[k_i] == 1
        append!(k_t, D[k_i])
      end
    end
    Ko[join(sort(k_t))] = v - 1
  end
  
  @_ O                   |>
     map(collect.(_),__) |> 
     map(sort(_),__)     |>
     map(join(_),__)     |>
     map(Ko[_],__)       |>
     join                |>
     parse(Int,__)
end

@_ D                           |>
   map(translate(_.i,_.o), __) |>
   sum

1046281